# ✏️ Basic Setting

## 🔎 Data mount

보안상의 이유로 스킵

## 🔎 Functions

In [ ]:
# check GPU
!nvidia-smi

Sat Oct 31 03:47:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    22W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/
#lgbm with gpu on colab
!git clone https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install
!pip3 uninstall scikit-learn
!pip3 install scikit-learn==0.21.3

/content
Cloning into 'LightGBM'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 19977 (delta 1), reused 2 (delta 0), pack-reused 19970
Receiving objects: 100% (19977/19977), 15.82 MiB | 13.15 MiB/s, done.
Resolving deltas: 100% (14566/14566), done.
/content/LightGBM
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found versi

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average='macro'):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
class preprocessing_test:
  
  def __init__(self, df, GPU=True):
    self.df = df
    self.device = 'gpu' if GPU == True else 'cpu'
    self.tree_method = 'gpu_hist' if GPU==True else 'auto'
    X = self.df.iloc[:, 1:].values; y = self.df.iloc[:, 0].values
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)

  def LIFT20 (self,original_df, y_test, y_pred):
    
    LIFT_value = []

    total_data = len(original_df)
    predict_df = pd.DataFrame(data = y_pred, columns=['가맹점0','가맹점1','가맹점2','가맹점3','가맹점4','가맹점5',
                                   '가맹점6','가맹점7','가맹점8','가맹점9','가맹점10'])
    predict_df['y_test'] = y_test
    for classes in range(11):
        df = predict_df.sort_values(by=['가맹점%s'%classes], ascending=False)
        df = df[:int(len(predict_df)*0.2)]
        
        denominator = len(original_df[original_df['MRC_ID_DI']==int(classes)])/total_data
        numerator = len(df[df['y_test'] == classes])/int(len(predict_df)*0.2)
        LIFT_value.append(numerator/denominator)
    
    
    #결과 출력부
    for i in range(11):
      print('\033[36m' + '가맹점 %d_LIFT =\t\t'%i + str(round(LIFT_value[i], 4)) + '\033[0m') 
    print('\033[33m' + 'Mean_LIFT =\t\t' + str(round(np.mean(LIFT_value), 4)) + '\033[0m', sep='\n')

        
    #print(LIFT_value, np.mean(LIFT_value))
  
  def multiclass_roc_auc_score(self,y_test, y_pred_proba, average=None): #average='macro' when you wanna get mean
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    #y_pred = lb.transform(y_pred)   #if y_pred_proba is not a probability
    roc_auc = roc_auc_score(y_test, y_pred_proba, average=average)

    #결과 출력부
    for i in range(11):
      print('\033[36m'+'가맹점 %d_rocauc =\t'%i + str(round(roc_auc[i], 4))+'\033[0m') 
    print('\033[33m' + 'Mean_rocauc =\t\t' + str(round(np.mean(roc_auc), 4)) + '\033[0m', sep='\n')

  def lgbm_test(self):


    lgbm = LGBMClassifier(device=self.device, seed=1234)
    %time lgbm.fit(self.X_train, self.y_train)
    y_pred = lgbm.predict_proba(self.X_test)

    #lgbm 결과 출력
      #roc_auc
    print('\033[1m' + '\033[91m'+'Result of lgbm_model'+'\033[0m')
    self.multiclass_roc_auc_score(self.y_test, y_pred)
      #LIFT@20
    self.LIFT20(self.df, self.y_test,y_pred)

  def xgb_test(self):
    #X = self.df.iloc[:, 1:].values; y = self.df.iloc[:, 0].values
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)
    
    xgb = XGBClassifier(random_state=1234, tree_method=self.tree_method)
    %time xgb.fit(self.X_train, self.y_train, verbose=True)
    y_pred = xgb.predict_proba(self.X_test)

    #xgb 결과 출력
      #roc_auc
    print('\033[1m'+'\033[91m'+'Result of xgb_model'+'\033[0m')
    self.multiclass_roc_auc_score(self.y_test, y_pred)
      #LIFT@20
    self.LIFT20(self.df, self.y_test,y_pred)

In [ ]:
def concat_array(X_cat, X_num):
    X = np.concatenate((X_cat, X_num), axis=1)
    return X

# ✏️ Hyperparameter tuning (GridSearchCV)

## 🔎  FA (lgbm, 28+79 dim)

In [ ]:
from factor_analyzer import FactorAnalyzer

fa = FactorAnalyzer(rotation='varimax',  n_factors = 79)
%time fa.fit(X_num)

# Change column values with FA
%time X_fa = fa.transform(X_num)

/usr/local/lib/python3.6/dist-packages/factor_analyzer/factor_analyzer.py:536: UserWarning: Failed to converge: b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
  warnings.warn('Failed to converge: {}'.format(res.message))


CPU times: user 3min 7s, sys: 3min 46s, total: 6min 54s
Wall time: 3min 30s
CPU times: user 1.14 s, sys: 291 ms, total: 1.43 s
Wall time: 917 ms


In [ ]:
X = concat_array(X_cat,X_fa)

In [ ]:
X_cat.shape, X_fa.shape, X.shape

((285006, 28), (285006, 79), (285006, 107))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score

lgbm = LGBMClassifier(
    device='gpu' 
)

In [ ]:
%time lgbm.fit(X_train, y_train)

CPU times: user 1min 6s, sys: 1.06 s, total: 1min 7s
Wall time: 36.7 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [ ]:
y_pred = lgbm.predict_proba(X_test)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
multiclass_roc_auc_score(y_test, y_pred, average='macro')

0.8590869903373041

In [ ]:
import time
start = time.time()

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

lgbm = LGBMClassifier(
    device='gpu' 
)

lgbm_params = {
    'learning_rate' : [0.01, 0.03, 0.07, 0.1],
    'n_estimators': [n for n in range(100,300,20)]
}

lgbm_grid = GridSearchCV(lgbm, param_grid=lgbm_params, scoring=f1, cv=5, verbose=1, n_jobs=-1)
lgbm_grid.fit(X_train, y_train)

print("Best Score : {}".format(lgbm_grid.best_score_))
print("Best Params : {}".format(lgbm_grid.best_params_))
print("time :", time.time() - start)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 184.0min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 188.5min finished


Best Score : 0.18189558284482202
Best Params : {'learning_rate': 0.1, 'n_estimators': 220}
time : 11376.133738517761


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score

lgbm = LGBMClassifier(
    device='gpu',
    learning_rate = 0.1,
    n_estimatros = 220
)

In [ ]:
%time lgbm.fit(X_train, y_train)

CPU times: user 1min 3s, sys: 917 ms, total: 1min 4s
Wall time: 32.8 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_estimatros=220,
               n_jobs=-1, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_pred = lgbm.predict_proba(X_test)

In [ ]:
multiclass_roc_auc_score(y_test, y_pred, average='macro')

0.8547472717871867

## 🔎 FA (lgbm, 28+79 dim)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score

lgbm = LGBMClassifier(
    device='gpu' 
)

In [ ]:
import time
start = time.time()

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

lgbm = LGBMClassifier(
    device='gpu' 
)

lgbm_params = {
    'learning_rate' : [0.01, 0.03, 0.07, 0.1, 0.2],
    'n_estimators': [n for n in range(100,300,20)]
}

lgbm_grid = GridSearchCV(lgbm, param_grid=lgbm_params, scoring=f1, cv=5, verbose=1, n_jobs=-1)
lgbm_grid.fit(X_train, y_train)

print("Best Score : {}".format(lgbm_grid.best_score_))
print("Best Params : {}".format(lgbm_grid.best_params_))
print("time :", time.time() - start)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 58.7min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 260.6min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 329.9min finished


Best Score : 0.18190726532616303
Best Params : {'learning_rate': 0.1, 'n_estimators': 160}
time : 19859.976344347


## 🔎  lgbm, full dim (max_depth, min_child_samples, num_leaves)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
import time
start = time.time()

from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score, average='macro')

lgbm = LGBMClassifier(
    device='gpu', learning_rate=0.1, n_estimators=160
)

lgbm_params = {
'max_depth' : [10, 20, 30],
'min_child_samples' : [5, 10, 15, 20],
'num_leaves':  [20, 30, 40]
}

lgbm_grid = GridSearchCV(lgbm, param_grid=lgbm_params, scoring=f1, cv=4, verbose=2)
lgbm_grid.fit(X_train, y_train)

print("Best Score : {}".format(lgbm_grid.best_score_))
print("Best Params : {}".format(lgbm_grid.best_params_))
print("time :", time.time() - start)

Fitting 4 folds for each of 36 candidates, totalling 144 fits
[CV] max_depth=10, min_child_samples=5, num_leaves=20 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . max_depth=10, min_child_samples=5, num_leaves=20, total=  47.9s
[CV] max_depth=10, min_child_samples=5, num_leaves=20 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   47.9s remaining:    0.0s


[CV] . max_depth=10, min_child_samples=5, num_leaves=20, total=  48.2s
[CV] max_depth=10, min_child_samples=5, num_leaves=20 ................
[CV] . max_depth=10, min_child_samples=5, num_leaves=20, total=  48.5s
[CV] max_depth=10, min_child_samples=5, num_leaves=20 ................
[CV] . max_depth=10, min_child_samples=5, num_leaves=20, total=  48.2s
[CV] max_depth=10, min_child_samples=5, num_leaves=30 ................
[CV] . max_depth=10, min_child_samples=5, num_leaves=30, total= 1.0min
[CV] max_depth=10, min_child_samples=5, num_leaves=30 ................
[CV] . max_depth=10, min_child_samples=5, num_leaves=30, total= 1.0min
[CV] max_depth=10, min_child_samples=5, num_leaves=30 ................
[CV] . max_depth=10, min_child_samples=5, num_leaves=30, total= 1.0min
[CV] max_depth=10, min_child_samples=5, num_leaves=30 ................
[CV] . max_depth=10, min_child_samples=5, num_leaves=30, total= 1.0min
[CV] max_depth=10, min_child_samples=5, num_leaves=40 ................
[CV] .

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 155.5min finished


Best Score : 0.18187812022200148
Best Params : {'max_depth': 10, 'min_child_samples': 20, 'num_leaves': 30}
time : 9389.721680879593


Test 

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score

lgbm = LGBMClassifier(
    device='gpu',
    learning_rate = 0.1,
    n_estimatros = 160,
    max_depth = 10,
    num_leaves = 30,
    min_child_samples = 20
)

In [ ]:
%time lgbm.fit(X_train, y_train)

CPU times: user 1min 29s, sys: 1.33 s, total: 1min 30s
Wall time: 47.2 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.1,
               max_depth=10, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_estimatros=160,
               n_jobs=-1, num_leaves=30, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_pred = lgbm.predict_proba(X_test)

In [ ]:
multiclass_roc_auc_score(y_test, y_pred, average='macro')

0.8646291246453544